# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
from datetime import datetime
import chromadb
from chromadb.utils import embedding_functions
from typing import List, Dict, TypedDict, Optional
from dotenv import load_dotenv
from tavily import TavilyClient

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage, BaseMessage
from lib.tooling import tool
from lib.vector_db import VectorStoreManager
from lib.memory import LongTermMemory, MemoryFragment, MemorySearchResult
from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Run


In [5]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CHROMA_OPENAI_API_KEY = os.getenv("CHROMA_OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Setup ChromaDB

In [6]:
chroma_client = chromadb.PersistentClient(path="chromadb")
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(api_key=CHROMA_OPENAI_API_KEY)
collection = chroma_client.get_collection(name="udaplay", embedding_function=embedding_fn)


### Setup Tavily

In [7]:
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [8]:
def retrieve_game(query: str):
    """
    Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry. 

    Returns results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """

    results = collection.query(
        query_texts=[query],
        n_results=5
    )

    docs = results['documents'][0]
    metadatas = results['metadatas'][0]
    
    retrieved = []
    for doc, meta in zip(docs, metadatas):
        retrieved.append({
            "Name": meta.get("Name"),
            "Platform": meta.get("Platform"),
            "YearOfRelease": meta.get("YearOfRelease"),
            "Description": meta.get("Description")
        })
    return str(retrieved)

retrieve_game_tool = tool(
    func=retrieve_game,
    name="retrieve_game",
    description="Search the video game information in the local database."
)

  

#### Evaluate Retrieval Tool

In [9]:
def evaluate_retrieval(question: str, retrieved_docs: str):
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question. 
    args: 
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    
    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """

    # Use a separate LLM call to evaluate
    llm = LLM(
        model="gpt-4o-mini",
        api_key=OPENAI_API_KEY
    )

    prompt = f"""
    Your task is to evaluate if the documents are enough to respond the query.
    Give a detailed explanation, so it's possible to take an action to accept it or not.
    Question: {question}
    Retrieved: {retrieved_docs}

    Return a JSON with "useful" (boolean) and "description" (string).
    """

    response = llm.invoke([
        UserMessage(content=prompt)
    ])
    return response.content

evaluate_retrieval_tool = tool(
    func=evaluate_retrieval,
    name="evaluate_retrieval",
    description="Evaluate if retrieved documents are sufficient "
)



#### Game Web Search Tool

In [10]:
def game_web_search(query: str, search_depth: str = "advanced"):
    """
    Search the web using Tavily API
    args:
        query (str): Search query
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)
    """

    # Perform the search
    search_result = tavily_client.search(
        query=query,
        saerch_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )

    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }

    return str(formatted_results)

game_web_search_tool = tool(
    func=game_web_search,
    name="game_web_search",
    description="Search the web for game information when local database is insufficient."
)

### Agent

In [11]:
tools = [retrieve_game_tool, evaluate_retrieval_tool, game_web_search_tool]
agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
        "You are UdaPlay, an AI Research Agent for the video game industry. "
        "Your goal is to answer user questions comprehensively. "
        "1. First, search your internal memory (vector DB) using `retrieve_game`. "
        "2. Evaluate if the retrieved info is sufficient using `evaluate_retrieval`. "
        "3. If not sufficient, search the web using `game_web_search`. "
        "4. Always cite your sources. If you found it in the local database, cite 'Local DB'. "
        "If you found it on the web, cite the URL or 'Web Search'. "
        "5. Structure your answer clearly."
    ),
    tools=tools
)

In [12]:
# print agent trace
def print_agent_response(response):
    messages = response.get_final_state()["messages"]
    
    print("AGENT Trace")
    print("===============")
    for msg in messages:
        role = msg.role.upper()
        if role == "SYSTEM":
            continue
        print(f"\n[{role}]")
        # tool Calls (in AI message)
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            for tc in msg.tool_calls:
                print(f"\n Tool Call: {tc.function.name}({tc.function.arguments})")
        # Content
        if msg.content:  
            print(f"{msg.content}")
    print("===============")

# helper method to run query
def run_query(query, session_id):
    print(f"\n --- User Query: {query} ---")
    response = agent.invoke(
        query=query,
        session_id=session_id
    )
    print_agent_response(response)


In [13]:
# Test Queries
run_query('Which one was the first 3D platformer Mario Game?', "first_session")
run_query('When was it released?', "first_session")
run_query('Was Mortal Kombat X realeased for Playstation 5?', "session_2")


 --- User Query: Which one was the first 3D platformer Mario Game? ---
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
AGENT Trace

[USER]
Which one was the first 3D platformer Mario Game?

[ASSISTANT]

 Tool Call: retrieve_game({"query":"first 3D platformer Mario Game"})

[TOOL]
"[{'Name': 'Super Mario 64', 'Platform': 'Nintendo 64', 'YearOfRelease': 1996, 'Description': \"A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.\"}, {'Name': 'Super Mario World', 'Platform': 'Super Nintendo Entertainment System (SNES)', 'YearOfRelease': 1990, 'Description': 'A classic platformer where Mario embarks on a quest to save Princess Toa

### (Optional) Advanced

In [7]:
vector_store_manager = VectorStoreManager(openai_api_key=OPENAI_API_KEY)
ltm = LongTermMemory(vector_store_manager)
llm = LLM(
    model="gpt-4o-mini",
    api_key=OPENAI_API_KEY
)

### AGENT STATE - Define Schema

In [8]:
class AgentState(TypedDict):
    input_query: str
    user_id: str
    context_games: Optional[str]
    context_memory: Optional[str]
    context_web: Optional[str]
    is_sufficient: bool
    final_answer: str

### Define Steps

### Step: Retrieve memory

In [9]:
def retrieve_memory_step(state: AgentState) -> AgentState:
    """Check Long Term Memory for user context"""
    print(f"   [Step] Check Long Term Memory for user context...{state["input_query"],}")
    print(f"   [Step] Checking Memory for user: {state['user_id']}")
    results = ltm.search(
        query_text=state["input_query"],
        owner=state["user_id"],
        limit=2
    )
    memories = [f"- {m.content}" for m in results.fragments]
    print(f"   [Step] prepare return info context memory... {memories}")
    context_str = "\n".join(memories) if memories else "No relevant personal memories found."
    print(f"   [Step] prepare return info context memory...")
    return {
        **state,
        "context_memory": context_str
    }

### Step: Retreive game

In [10]:
def retrieve_game_step(state: AgentState) -> AgentState:
    print(f"   [step] Retrieving Game Info...")
    results = collection.query(
        query_texts=[state["input_query"]],
        n_results=3
    )
    
    docs = results['documents'][0]
    metadatas = results['metadatas'][0]
        
    game_info = []
    for doc, meta in zip(docs, metadatas):
        name = str(meta.get('Name', 'Unknown'))
        platform = str(meta.get('Platform', 'Unknown'))
        year = str(meta.get('YearOfRelease', 'Unknown'))
        doc_str = str(doc)
        
        info = f"Game: {name} ({platform}, {year})\nDetails: {doc_str}"
        game_info.append(info)

    context_str = "\n---\n".join(game_info)

    return {
        **state,
        "context_games": context_str
    }


### Step: Evaluate state 

In [11]:
def evaluate_step(state: AgentState) -> AgentState:
    """Evaluate if retrieved info is sufficient"""
    print(f"   [Step] Evaluating Context....")

    prompt = f"""
    User Query: {state['input_query']}

    Retrieved Game Info:
    {state['context_games']}

    Retrieved User Memory:
    {state['context_memory']}
    
    Is this information sufficient to answer the user's query comfortably? 
    If the user asks about a game not listed here, or specific details missing here, answer NO.
    
    Respond with ONLY 'YES' or 'NO'.
    """

    response = llm.invoke([
        UserMessage(content=prompt)
    ])
    decision = response.content.strip().upper()

    is_sufficient = "YES" in decision
    print(f"   [Step] Evaluation Result: {is_sufficient}")
    
    return {
        **state,
        "is_sufficient": is_sufficient
    }

### Step: Web Search

In [12]:
def web_search_step(state: AgentState) -> AgentState:
    """Search web if local info is insufficient"""
    print(f"   [Step] Searching Web...")

    try:
        # Perform the search
        search_result = tavily_client.search(
            query=state['input_query'],
            saerch_depth="advanced",
            include_answer=True,
            include_raw_content=False,
            include_images=False
        )

        # Format the results
        results = search_result.get("results", [])
        web_context = "\n".join([f"- {r['content']}" for r in results[:3]])
    except Exception as e:
        web_context = f"Web search failed: {e}"    

    return {
        **state,
        "context_web": web_context
    }
    

### Step: Generate answer  

In [13]:
def generate_answer_step(state: AgentState) -> AgentState:
    """Generate final response using all available context"""
    print(f"   [Step] Generating Answer...")
    
    # Construct context based on what we have
    context_parts = []
    if state["context_memory"]:
        context_parts.append(f"User Context:\n{state['context_memory']}")
    
    if state["context_games"]:
        context_parts.append(f"Game Database Info:\n{state['context_games']}")
        
    if state.get("context_web"):
        context_parts.append(f"Web Search Results:\n{state['context_web']}")
        
    full_context = "\n\n".join(context_parts)
    
    system_prompt = "You are UdaPlay, an expert video game assistant. Answer the user query using the provided context."
    user_prompt = f"""
    Context:
    {full_context}
    
    Query: {state['input_query']}
    
    Answer:
    """
    
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        UserMessage(content=user_prompt)
    ])
    
    return {
        **state,
        "final_answer": response.content
    }

### Step: Memorize step 

In [14]:
def memorize_step(state: AgentState) -> AgentState:
    """Extract and save new user preferences if found"""
    print(f"   [Step] Checking for new memories...")
    
    prompt = f"""
    Analyze the user's query for any explicit user preferences or facts about themselves (e.g., "I like RPGs", "I own a PS5").
    Igore questions or general statements.
    
    Query: {state['input_query']}
    
    If a preference is found, output the preference content. 
    If nothing to memorize, output 'SKIP'.
    """
    
    response = llm.invoke([UserMessage(content=prompt)])
    content = response.content.strip()
    
    if content != "SKIP":
        ltm.register(MemoryFragment(content=content, owner=state['user_id']))
        print(f"   [Memory] Saved: {content}")
        
    return state

### Build & Connect State Machine Workflow

In [15]:
def build_agent_workflow():
    workflow = StateMachine[AgentState](AgentState)
    
    # Steps
    entry = EntryPoint[AgentState]()
    s_memory = Step[AgentState]("retrieve_memory", retrieve_memory_step)
    s_game = Step[AgentState]("retrieve_game", retrieve_game_step)
    s_eval = Step[AgentState]("evaluate", evaluate_step)
    s_web = Step[AgentState]("web_search", web_search_step)
    s_gen = Step[AgentState]("generate", generate_answer_step)
    s_memo = Step[AgentState]("memorize", memorize_step) # Running this after gen
    term = Termination[AgentState]()
    
    workflow.add_steps([entry, s_memory, s_game, s_eval, s_web, s_gen, s_memo, term])
    
    # Transitions
    
    # Entry -> Memory
    workflow.connect(entry, s_memory)
    
    # Memory -> Game Retrieval
    workflow.connect(s_memory, s_game)
    
    # Game -> Eval
    workflow.connect(s_game, s_eval)
    
    # Eval -> Web OR Generate using condition logic
    def check_eval(state: AgentState):
        if state["is_sufficient"]:
            return s_gen
        else:
            return s_web
            
    workflow.connect(s_eval, [s_gen, s_web], check_eval)
    
    # Web -> Generate
    workflow.connect(s_web, s_gen)
    
    # Generate -> Memorize
    workflow.connect(s_gen, s_memo)
    
    # Memorize -> Termination
    workflow.connect(s_memo, term)
    
    return workflow

### Run the workflow

In [16]:
workflow = build_agent_workflow()



In [18]:
# Session 1: User states preference
print("\n\n=== RUN 1: Preference Sharing ===")
initial_state = {
    "input_query": "I love open-world games like GTA.",
    "user_id": "test_user_1",
    "context_games": None,
    "context_memory": None,
    "context_web": None,
    "is_sufficient": False,
    "final_answer": ""
}

result_run = workflow.run(initial_state)
print(f"Final Answer: {result_run.get_final_state()['final_answer']}")



=== RUN 1: Preference Sharing ===
[StateMachine] Starting: __entry__
   [Step] Check Long Term Memory for user context...('I love open-world games like GTA.',)
   [Step] Checking Memory for user: test_user_1
   [Step] prepare return info context memory... ['- open-world games like GTA']
   [Step] prepare return info context memory...
[StateMachine] Executing step: retrieve_memory
   [step] Retrieving Game Info...
[StateMachine] Executing step: retrieve_game
   [Step] Evaluating Context....
   [Step] Evaluation Result: True
[StateMachine] Executing step: evaluate
   [Step] Generating Answer...
[StateMachine] Executing step: generate
   [Step] Checking for new memories...
   [Memory] Saved: open-world games like GTA
[StateMachine] Executing step: memorize
[StateMachine] Terminating: __termination__
Final Answer: If you love open-world games like Grand Theft Auto, you might enjoy **Marvel's Spider-Man**. It's an open-world superhero game that allows you to explore New York City while ta

In [20]:
print("\n\n=== RUN 2: RAG Query ===")
initial_state_2 = {
    "input_query": "When was San Andreas released?",
    "user_id": "test_user_1",
    "context_games": None,
    "context_memory": None,
    "context_web": None,
    "is_sufficient": False,
    "final_answer": ""
}
result_run_2 = workflow.run(initial_state_2)
print(f"Final Answer: {result_run_2.get_final_state()['final_answer']}")



=== RUN 2: RAG Query ===
[StateMachine] Starting: __entry__
   [Step] Check Long Term Memory for user context...('When was San Andreas released?',)
   [Step] Checking Memory for user: test_user_1
   [Step] prepare return info context memory... ['- open-world games like GTA', '- open-world games like GTA']
   [Step] prepare return info context memory...
[StateMachine] Executing step: retrieve_memory
   [step] Retrieving Game Info...
[StateMachine] Executing step: retrieve_game
   [Step] Evaluating Context....
   [Step] Evaluation Result: True
[StateMachine] Executing step: evaluate
   [Step] Generating Answer...
[StateMachine] Executing step: generate
   [Step] Checking for new memories...
[StateMachine] Executing step: memorize
[StateMachine] Terminating: __termination__
Final Answer: Grand Theft Auto: San Andreas was released on PlayStation 2 in 2004.


In [21]:
print("\n\n=== RUN 3: Web Search Query ===")
initial_state_3 = {
    "input_query": "What is the latest Cyberpunk 2077 DLC?",
    "user_id": "test_user_1",
    "context_games": None,
    "context_memory": None,
    "context_web": None,
    "is_sufficient": False,
    "final_answer": ""
}
result_run_3 = workflow.run(initial_state_3)
print(f"Final Answer: {result_run_3.get_final_state()['final_answer']}")



=== RUN 3: Web Search Query ===
[StateMachine] Starting: __entry__
   [Step] Check Long Term Memory for user context...('What is the latest Cyberpunk 2077 DLC?',)
   [Step] Checking Memory for user: test_user_1
   [Step] prepare return info context memory... ['- open-world games like GTA', '- open-world games like GTA']
   [Step] prepare return info context memory...
[StateMachine] Executing step: retrieve_memory
   [step] Retrieving Game Info...
[StateMachine] Executing step: retrieve_game
   [Step] Evaluating Context....
   [Step] Evaluation Result: False
[StateMachine] Executing step: evaluate
   [Step] Searching Web...
[StateMachine] Executing step: web_search
   [Step] Generating Answer...
[StateMachine] Executing step: generate
   [Step] Checking for new memories...
[StateMachine] Executing step: memorize
[StateMachine] Terminating: __termination__
Final Answer: The latest DLC for Cyberpunk 2077 is called "Phantom Liberty," which was released on September 25, 2023. This expansi